In [1]:
import boto3
import pandas as pd
from ete3 import NCBITaxa

In [3]:
s3 = boto3.resource('s3')
client = boto3.client('s3')
bucket = s3.Bucket("czbiohub-mosquito")

In [171]:
bucket_name = "czbiohub-mosquito"
# First list the folders (there is a limit of 1000 files output by AWS by default)
list_of_folders = [client.list_objects(Bucket=bucket_name, Prefix=x["Prefix"]+"blast_nt.m9") \
                   for x in client.list_objects(Bucket=bucket_name, Prefix="contigs/", Delimiter="/")["CommonPrefixes"]]
list_of_folders = ["s3://"+bucket_name+"/"+x["Prefix"] for x in list_of_folders if "Contents" in x.keys()]



In [234]:
filenames = pd.DataFrame(list_of_folders, columns=["blast_nt"])
filenames = filenames.assign(filtered_blast_nt=filenames["blast_nt"].str.replace("contigs", "contig_quality").str.replace("_nt.m9", "_nt_filtered.m9"))
filenames = filenames.assign(lca_nt=filenames["filtered_blast_nt"].str.replace("blast_nt_filtered.m9", "lca_nt.m9"))

The nt hits of contigs from each sample are filtered with ident_cutoff=0.5 and align_len_cutoff=0.2

In [235]:
commands = filenames.apply(lambda x: "python lca_analysis.py --blast_type nt --fpath "+x.iloc[0]+\
                " --filtered_blast_path "+x.iloc[1]+" --outpath "+x.iloc[2]+\
                " --ident_cutoff 0.5 --align_len_cutoff 0.5", axis=1)


In [236]:
commands.to_csv("lca_analysis_commands", index=False)

In [187]:
%%bash
parallel -a lca_analysis_commands -j 16



Process is interrupted.
